### 라이브러리

In [19]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

### API 키

In [20]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

### PDF 로드 / 임베딩

In [21]:
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.split_documents(pages)



In [27]:
# 3. OpenAI 임베딩 & 벡터스토어 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 4. Claude LLM
llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    temperature=0,
    api_key=ANTHROPIC_API_KEY,
)

# 5. Prompt (LangChain Hub or 직접 템플릿)
try:
    prompt = hub.pull("rlm/rag-prompt")
except:
    from langchain_core.prompts import PromptTemplate
    prompt = PromptTemplate.from_template(
        "다음 맥락을 기반으로 질문에 답하세요.\n\nContext: {context}\nQuestion: {question}"
    )

# 6. RAG 체인
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\KDP014\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
questions = [
    "특정암 ii란 뭐야?",
    "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?",
    "통합암진단비ii 는 보장범위가 어떻게 돼?",
    "통합전이암진단비의 보장범위는 어떻게 돼?",
    "보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?",
    "신재진단암진단비ii 에서 수술이란 뭐야?",
    "보험금청구 했는데 언제 지급되니?"
]

for idx, q in enumerate(questions, start=1):
    print("\n" + "=" * 50)
    print(f"[{idx}] 🧾 질문: {q}")
    print("=" * 50)

    # LLM 단독 응답
    llm_answer = llm.invoke(q)
    print("\n💬 [LLM 단독 응답]:")
    print(llm_answer)

    # RAG 기반 응답
    rag_answer = rag_chain.invoke(q)
    print("\n📚 [RAG 기반 응답]:")
    print(rag_answer)

    print("\n" + "-" * 50)



[1] 🧾 질문: 특정암 ii란 뭐야?

💬 [LLM 단독 응답]:
content='특정암 II는 한국 중앙암등록본부에서 지정한 암 분류 체계 중 하나입니다. 이 분류에 속하는 암은 다음과 같습니다:\n\n1. 식도암\n2. 간암\n3. 담낭 및 기타 담도암\n4. 췌장암\n5. 후두암\n6. 폐암\n7. 흑색종을 제외한 피부암\n8. 중추신경계 암\n9. 갑상선암\n10. 호지킨림프종\n11. 비호지킨림프종\n12. 다발성골수종\n13. 백혈병\n14. 기타 암\n\n이 분류는 암의 발생 부위와 종류에 따라 구분되며, 암 통계 및 연구에 활용됩니다. 특정암 II에 속하는 암들은 상대적으로 발생 빈도가 높거나 예후가 좋지 않은 경우가 많습니다. 이러한 분류를 통해 암의 예방, 조기 진단, 치료 및 관리 전략을 수립하는 데 도움을 줍니다.' additional_kwargs={} response_metadata={'id': 'msg_01XMvh9r76hNieW5aET8RyRm', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 20, 'output_tokens': 361, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-opus-20240229'} id='run--85acc4d2-49e8-4791-8378-6d6d7fad354b-0' usage_metadata={'input_tokens': 20, 'output_tokens': 361, 'total_tokens': 381, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}}

📚 [RAG 기반 응답]:


In [32]:
# 결과 저장 파일명
output_file = "rag_results.txt"

# 파일 열기 (기존 내용 덮어쓰기)
with open(output_file, "w", encoding="utf-8") as f:
    for idx, q in enumerate(questions, start=1):
        # 구분선 및 질문 번호
        header = "\n" + "=" * 50 + f"\n[{idx}] 🧾 질문: {q}\n" + "=" * 50 + "\n"
        print(header)
        f.write(header + "\n")

        # LLM 단독 응답
        llm_answer = llm.invoke(q)
        llm_output = f"\n💬 [LLM 단독 응답]:\n{llm_answer}\n"
        print(llm_output)
        f.write(llm_output + "\n")

        # RAG 기반 응답
        rag_answer = rag_chain.invoke(q)
        rag_output = f"\n📚 [RAG 기반 응답]:\n{rag_answer}\n"
        print(rag_output)
        f.write(rag_output + "\n")

        # 하위 구분선
        footer = "\n" + "-" * 50 + "\n"
        print(footer)
        f.write(footer + "\n")

print(f"\n결과가 '{output_file}' 파일로 저장되었습니다.")



[1] 🧾 질문: 보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?


💬 [LLM 단독 응답]:
content='보험 약관에 따라 다르겠지만, 일반적으로 표적항암방사선 치료비는 보험가입금액의 100%를 지급하는 경우가 많습니다. 다만 몇 가지 고려해야 할 사항이 있습니다.\n\n1. 보험 약관 확인: 가입한 보험의 약관을 꼼꼼히 살펴보시고, 표적항암방사선 치료에 대한 보장 내용과 지급 조건을 확인하세요.\n\n2. 면책기간: 보험 가입 후 일정 기간 동안(주로 90일 또는 180일)은 보험금을 지급하지 않는 면책기간이 있을 수 있습니다. 보험 가입 후 8개월이 지났다면 면책기간이 지난 것으로 보입니다.\n\n3. 진단 확인: 표적항암방사선 치료를 받기 위해서는 의료기관에서 암 진단을 받아야 합니다. 보험회사에서 요구하는 진단서 및 관련 서류를 제출해야 합니다.\n\n4. 치료 확인: 표적항암방사선 치료를 받았다는 증빙 서류(진료비 영수증, 치료 기록 등)를 보험회사에 제출해야 합니다.\n\n위 사항을 모두 충족한다면 보험가입금액의 100%를 받을 수 있을 것으로 보입니다. 하지만 개별 상황에 따라 다를 수 있으므로, 가입한 보험사에 문의하여 정확한 정보를 확인하시는 것이 좋습니다.' additional_kwargs={} response_metadata={'id': 'msg_01KZLwTXrQDVfKE5i37WoLqi', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 58, 'output_tokens': 541, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-o

In [30]:
questions = [
    "통합암진단비ii 는 보장범위가 어떻게 돼?",
    "통합전이암진단비의 보장범위는 어떻게 돼?",
]

for idx, q in enumerate(questions, start=1):
    print("\n" + "=" * 50)
    print(f"[{idx}] 🧾 질문: {q}")
    print("=" * 50)

    # LLM 단독 응답
    llm_answer = llm.invoke(q)
    print("\n💬 [LLM 단독 응답]:")
    print(llm_answer)

    # RAG 기반 응답
    rag_answer = rag_chain.invoke(q)
    print("\n📚 [RAG 기반 응답]:")
    print(rag_answer)

    print("\n" + "-" * 50)



[1] 🧾 질문: 통합암진단비ii 는 보장범위가 어떻게 돼?

💬 [LLM 단독 응답]:
content='통합암진단비II는 일반적으로 다음과 같은 보장 범위를 가지고 있습니다:\n\n1. 일반암: 기타피부암 및 갑상선암 이외의 악성신생물(암)에 대해 보장합니다.\n\n2. 특정소액암: 유방암, 자궁경부암, 자궁체부암, 전립선암, 방광암 등 특정 부위의 암에 대해 보장합니다.\n\n3. 기타피부암: 악성흑색종을 제외한 피부의 악성신생물(암)에 대해 보장합니다.\n\n4. 갑상선암: 갑상선의 악성신생물(암)에 대해 보장합니다.\n\n5. 상피내암: 제자리암(상피내암)에 대해 보장합니다.\n\n6. 경계성종양: 행동양식 불명 또는 미상의 신생물에 대해 보장합니다.\n\n다만, 보험사마다 상품의 구성과 명칭, 보장 범위 등이 조금씩 다를 수 있으므로, 가입하시려는 보험사의 상품설명서와 약관을 꼭 확인하시는 것이 좋습니다. 또한, 보험 가입 전에는 자신의 건강 상태와 필요에 맞는 보장 내용인지 잘 살펴보시고, 설계사 또는 상담원과 충분히 상의하시는 것을 추천드립니다.' additional_kwargs={} response_metadata={'id': 'msg_01Hc5cFPbqTm4L9de1TrcELF', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 32, 'output_tokens': 480, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-opus-20240229'} id='run--2fbcea72-f4e4-4573-87a2-d5f123af4b45-0' usage_metadata={'input_tokens': 

In [31]:
questions = [
    "보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?",
    "신재진단암진단비ii 에서 수술이란 뭐야?",
    "보험금청구 했는데 언제 지급되니?"
]

for idx, q in enumerate(questions, start=1):
    print("\n" + "=" * 50)
    print(f"[{idx}] 🧾 질문: {q}")
    print("=" * 50)

    # LLM 단독 응답
    llm_answer = llm.invoke(q)
    print("\n💬 [LLM 단독 응답]:")
    print(llm_answer)

    # RAG 기반 응답
    rag_answer = rag_chain.invoke(q)
    print("\n📚 [RAG 기반 응답]:")
    print(rag_answer)

    print("\n" + "-" * 50)



[1] 🧾 질문: 보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?

💬 [LLM 단독 응답]:
content='보험 약관에 따라 다르겠지만, 일반적으로 표적항암방사선 치료비는 보험가입금액의 100%를 지급하는 경우가 많습니다. 다만 몇 가지 고려해야 할 사항이 있습니다.\n\n1. 보험 약관 확인: 가입한 보험의 약관을 꼼꼼히 살펴보세요. 약관에 표적항암방사선 치료비에 대한 보장 내용과 지급 기준이 명시되어 있을 것입니다.\n\n2. 면책기간: 보험 가입 후 일정 기간 동안은 보험금을 지급하지 않는 면책기간이 있을 수 있습니다. 보통 암 진단의 경우 90일 정도의 면책기간이 적용되는 경우가 많습니다.\n\n3. 대기기간: 보험 가입 후 일정 기간이 지나야 보장이 개시되는 대기기간이 있을 수 있습니다.\n\n4. 진단 시기: 보험 가입 이후에 암으로 진단받은 경우에 한해 보험금이 지급됩니다.\n\n5. 치료 방법: 약관에 명시된 표적항암방사선 치료에 해당하는 치료를 받은 경우에만 보험금을 받을 수 있습니다.\n\n따라서 보험 약관을 자세히 확인하고, 필요하다면 보험사에 문의하여 정확한 내용을 파악하는 것이 좋습니다. 보험금 청구 시에는 진단서, 치료 내역 등 관련 서류를 제출해야 합니다.' additional_kwargs={} response_metadata={'id': 'msg_01UDDM42zR5i6VK4L2zraVfW', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 58, 'output_tokens': 546, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-

In [33]:
questions = [
    "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?",   # 2번 질문
    "통합암진단비ii 는 보장범위가 어떻게 돼?",                                           # 3번 질문
    "통합전이암진단비의 보장범위는 어떻게 돼?"                                          # 4번 질문
]

# 결과 저장 파일명
output_file = "rag_results.txt"

# 파일 열기 (기존 내용 덮어쓰기)
with open(output_file, "w", encoding="utf-8") as f:
    for idx, q in enumerate(questions, start=1):
        # 구분선 및 질문 번호
        header = "\n" + "=" * 50 + f"\n[{idx}] 🧾 질문: {q}\n" + "=" * 50 + "\n"
        print(header)
        f.write(header + "\n")

        # LLM 단독 응답
        llm_answer = llm.invoke(q)
        llm_output = f"\n💬 [LLM 단독 응답]:\n{llm_answer}\n"
        print(llm_output)
        f.write(llm_output + "\n")

        # RAG 기반 응답
        rag_answer = rag_chain.invoke(q)
        rag_output = f"\n📚 [RAG 기반 응답]:\n{rag_answer}\n"
        print(rag_output)
        f.write(rag_output + "\n")

        # 하위 구분선
        footer = "\n" + "-" * 50 + "\n"
        print(footer)
        f.write(footer + "\n")

print(f"\n결과가 '{output_file}' 파일로 저장되었습니다.")



[1] 🧾 질문: 유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?


💬 [LLM 단독 응답]:
content='유사암진단비III를 청구하기 위해서는 갑상선암에 대한 명확한 진단이 필요합니다. 일반적으로 갑상선암 진단을 위해 다음과 같은 검사들이 시행됩니다.\n\n1. 갑상선 초음파 검사: 갑상선의 크기, 모양, 결절 유무 등을 확인하는 영상 검사입니다.\n\n2. 갑상선 호르몬 검사: 혈액 검사를 통해 갑상선 기능을 평가합니다. (T3, T4, TSH 등)\n\n3. 갑상선 스캔: 방사성 요오드를 이용하여 갑상선의 기능과 결절의 특성을 평가하는 검사입니다.\n\n4. 세침흡인검사 (Fine Needle Aspiration, FNA): 의심되는 결절에서 세포를 채취하여 현미경으로 관찰함으로써 암 여부를 판단하는 검사입니다. 가장 중요한 확진 검사 중 하나입니다.\n\n5. 조직검사: 수술을 통해 의심되는 조직을 채취하여 병리학적으로 확진하는 검사입니다.\n\n보험사마다 유사암진단비III 청구를 위한 구체적인 요건이 다를 수 있으므로, 가입한 보험사에 문의하여 필요한 검사 결과와 진단서 등을 확인하는 것이 좋습니다. 일반적으로는 세침흡인검사나 조직검사를 통한 병리학적 확진이 필요할 것으로 예상됩니다.' additional_kwargs={} response_metadata={'id': 'msg_01SXGqzWBbTSJecCEo9q1VyX', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 55, 'output_tokens': 536, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-